In [205]:
import pandas as pd
import torch
import numpy as np
from torch import nn
import torch.utils.data as data
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [206]:
%run Model.ipynb
%run "../Machine Learning Data Gathering/Scripts/Moving Average + Bollinger bands.ipynb"

In [207]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

In [208]:
def get_all_data_from_folder(path):
    df = pd.read_csv(path)
    return df.values

In [209]:
single_stock = pd.read_csv("./Data/Stocks Data/Consumer Discretionary/Airbnb Inc. Class A Common Stock.csv", index_col="Date", parse_dates = True)
# single_stock = pd.read_csv("./Data/Stocks Data/Consumer Discretionary/Airbnb Inc. Class A Common Stock.csv")
# single_stock["Date"] = (single_stock["Date"].astype('datetime64[ns]')).apply(lambda x : x.timestamp())
# single_stock["Date"] = single_stock["Date"].astype('float64')
X = single_stock.drop("Close", axis = 1)
y = single_stock["Close"]
X.shape, y.shape


((850, 16), (850,))

In [210]:
# def create_dataset(input, output, period_in, period_out):
    
#     X, y = [], []
#     arr = np.array(input)
#     arr_two = np.array(output)
#     for i in range(len(arr) - period_in):
#         X.append(arr[i:i+period_in])
#         y.append(arr_two[i + 1 : i + period_out + 1])
#     return torch.Tensor(X), torch.Tensor(y)

def create_dataset(input, period_out):
    
    X, y = [], []
    
    for i in range(len(input) - period_out - 1):
        X.append(input[i: (i+period_out)])
        y.append(input[i + period_out])
    return torch.Tensor(np.array(X)), torch.Tensor(np.array(y))

def test_func(input_data, output_data, steps_for_input, steps_for_output):
    X,y = [], []
    
    for i in range(len(input_data)):
        end_x = i + steps_for_input
       # print(end_x)
        output_x = end_x + steps_for_output - 1
        if output_x > len(input_data): 
            break
        #print(output_data[end_x - 1 : output_x])
        seq_x, seq_y = input_data[i : end_x], output_data[end_x - 1 : output_x]
        X.append(seq_x), y.append(seq_y)
    
    # for i in range(X.shape[1]):
        
    #return scaler_test.fit_transform(X), scaler_test.transform(y)
        #print(seq_x)
        #print(seq_y)
    return np.array(X), np.array(y)

In [244]:
single_stock.dropna(inplace=True)
X = single_stock.drop("Close", axis = 1)

y = single_stock["Close"]
print(single_stock.shape)


X_calc, y_calc = test_func(X, y, 90, 28)
#X_calc, y_calc = create_dataset(single_stock.values, 14)

total_data = len(X)
test_split = round(0.80 * total_data)
X_train = torch.Tensor(X_calc[:test_split])
X_test = torch.Tensor(X_calc[test_split:])

y_train = torch.Tensor(y_calc[:test_split])
y_test = torch.Tensor(y_calc[test_split:])

test_scaler = StandardScaler()

mm_scaler = MinMaxScaler()

X_train = torch.Tensor(test_scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape))
X_test = torch.Tensor(test_scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape))

y_train = torch.Tensor(mm_scaler.fit_transform(y_train.reshape(-1, y_train.shape[-1])).reshape(y_train.shape))
y_test = torch.Tensor(mm_scaler.transform(y_test.reshape(-1, y_test.shape[-1])).reshape(y_test.shape))
print("Res", X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(836, 17)
Res torch.Size([669, 90, 16]) torch.Size([669, 28]) torch.Size([51, 90, 16]) torch.Size([51, 28])


### Custom Dataset testing

# Training the model

In [245]:
model = StocksPredictionModel(input_size=16, hidden_size=100, num_layers=1, num_classes = 28, batch_first=True)
model = model.to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001)
epochs = 1000

for epoch in range(epochs):
    model.train()
    X_train = X_train.cuda()
   
    y_pred = model(X_train)
  
    optimizer.zero_grad()
    y_pred = y_pred.to(device)
    y_train = y_train.to(device)
    loss = loss_fn(y_pred, y_train)

    loss.backward()

    optimizer.step()

    model.eval()

    with torch.inference_mode():
        
        test_pred = model(X_test)
       
        y_test = y_test.cuda()
        test_loss = loss_fn(test_pred, y_test)
        if epoch == 999:
                print(f"Epoch number: {epoch}")
                print(f"Test Loss is: {test_loss}")
                print(f"Train Loss is: {loss}")


Epoch number: 999
Test Loss is: 0.0094758914783597
Train Loss is: 0.0007634825888089836


In [249]:
test_data = single_stock
two = test_data["Close"]
test_data = test_data.drop("Close", axis = 1)

X_final, y_final = test_func(test_data.values, two, 90, 14)

#X_final = X_final[:,-14,:]
X_final = torch.Tensor(X_final[-1]).unsqueeze(0)

#X_final = torch.Tensor(X_final).unsqueeze(0)
X_final = torch.Tensor(test_scaler.fit_transform(X_final.reshape(-1, X_final.shape[-1])).reshape(X_final.shape))

print(X_final.shape)
# test_data = torch.Tensor(test_data.values)
# test_data = test_data.unsqueeze(0)

# test_data = torch.Tensor(test_scaler.fit_transform(test_data.reshape(-1, test_data.shape[-1])).reshape(test_data.shape))

with torch.no_grad():
    pred = model(X_final)

preds = pred.detach().cpu()
print(preds)
preds = mm_scaler.inverse_transform(preds)
print(preds)

torch.Size([1, 90, 16])
tensor([[0.5967, 0.5787, 0.5416, 0.5385, 0.4837, 0.4806, 0.4889, 0.4784, 0.5187,
         0.5394, 0.5730, 0.5847, 0.6331, 0.6721, 0.6819, 0.6705, 0.6885, 0.7302,
         0.7188, 0.7527, 0.7488, 0.7514, 0.7310, 0.7209, 0.7109, 0.6710, 0.5847,
         0.5758]])
[[156.9127035  154.66259082 150.03553583 149.65373858 142.81565089
  142.42591424 143.460738   142.15575546 147.18246019 149.76092043
  153.95494379 155.41067822 161.44467157 166.31580353 167.5426187
  166.11186214 168.36030219 173.56289652 172.13990095 176.37177024
  175.88154968 176.20453741 173.65999058 172.40349188 171.15455345
  166.1720841  155.41039573 154.30301347]]


# Saving the Model

In [250]:
MODEL_NAME = "01_stocks_predictions_model.pth"
torch.save(model.state_dict(), f"./Models/{MODEL_NAME}")

In [253]:
loaded_model = StocksPredictionModel(input_size=16, hidden_size=100, num_layers=1, num_classes = 28, batch_first=True)
loaded_model.load_state_dict(torch.load(f"./Models/{MODEL_NAME}"))

<All keys matched successfully>

In [254]:
loaded_model = loaded_model.to(device)
loaded_model.eval()
test_data = single_stock[-90:]
test_data = test_data.drop("Close", axis = 1)

test_data = torch.Tensor(test_data.values)
test_data = test_data.unsqueeze(0)

test_data = torch.Tensor(test_scaler.fit_transform(test_data.reshape(-1, test_data.shape[-1])).reshape(test_data.shape))

with torch.no_grad():
    pred = loaded_model(test_data)

preds = pred.detach().cpu()

preds = mm_scaler.inverse_transform(preds)
#print(preds)
# preds = np.pad(preds, ((0,0),(90,0)), 'constant')
# preds = np.squeeze(preds)


final_slice = single_stock[-90:]
final_slice = final_slice.reset_index()
preds = np.transpose(preds, (1, 0))
final = final_slice["Date"][-14:]
#print(final.values)


res = dict(zip(final, preds))

for k,v in res.items():
    print(k, " + " , v)



2024-04-10 00:00:00  +  [151.23917213]
2024-04-11 00:00:00  +  [150.87609341]
2024-04-12 00:00:00  +  [153.82813646]
2024-04-15 00:00:00  +  [151.61612248]
2024-04-16 00:00:00  +  [156.01899359]
2024-04-17 00:00:00  +  [159.66665563]
2024-04-18 00:00:00  +  [161.4343236]
2024-04-19 00:00:00  +  [166.89044337]
2024-04-22 00:00:00  +  [168.35436252]
2024-04-23 00:00:00  +  [170.75635704]
2024-04-24 00:00:00  +  [170.59046974]
2024-04-25 00:00:00  +  [175.78068664]
2024-04-26 00:00:00  +  [174.31825427]
2024-04-29 00:00:00  +  [174.68620218]
